In [1]:
import os
import configparser

# Loading configurations
configParser = configparser.RawConfigParser()   
configFilePath = r'configuration.txt'
configParser.read(configFilePath)

insert_amd_env_vars =  int(configParser.get('COMMON', 'insert_amd_env_vars'))
HSA_OVERRIDE_GFX_VERSION =  configParser.get('COMMON', 'HSA_OVERRIDE_GFX_VERSION')
ROCM_PATH =  configParser.get('COMMON', 'ROCM_PATH')

if(insert_amd_env_vars != 0):
    os.environ["HSA_OVERRIDE_GFX_VERSION"] = HSA_OVERRIDE_GFX_VERSION
    os.environ["ROCM_PATH"] = ROCM_PATH


import subprocess
import os
import pathlib
import configparser
import sqlite3 as sl
import cv2
from pydub import AudioSegment
import math
import pickle
import shutil
import time
import multiprocessing
from multiprocessing import Process,Queue
import itertools
from threading import Thread

start_time = time.time()    # To measure execution time in seconds


print("PLEASE EDIT configuration.txt BEFORE EXECUTION")
print(".wav files might be generated in path. The program will automatically delete them. If execuetion stops unexpectedly, please delete them yourself")



datasetPathVideo =  configParser.get('COMMON', 'datasetPathVideo')
datasetPathAudio =  configParser.get('extractAudio', 'datasetPathAudio')
p =  configParser.get('extractAudio', 'dbChunk')
ttwbdf =  int(configParser.get('extractAudio', 'time_to_wait_before_deleting_files'))
cuda =  int(configParser.get('COMMON', 'cuda'))
cpus =  int(configParser.get('COMMON', 'cpus'))
datasetPathDatabase =  configParser.get('COMMON', 'datasetPathDatabase') + '/dataset.db'
csv_folder =  configParser.get('train_s2fe_random_forest', 'csv_folder')
model_path =  configParser.get('train_s2fe_random_forest', 'model_path')
cpus_reg =  int(configParser.get('train_s2fe_random_forest', 'cpus_reg'))






PLEASE EDIT configuration.txt BEFORE EXECUTION
.wav files might be generated in path. The program will automatically delete them. If execuetion stops unexpectedly, please delete them yourself


In [2]:

import gc
from IPython.display import clear_output
import numpy as np



stacked_data_x = None
stacked_data_y = None
count_f = 0

while(True):
    count_f = count_f + 1
    if(os.path.exists(csv_folder + "/tree_reg_x" + str(count_f) + "_.csv")):

        
        loaded_data_x = np.loadtxt(csv_folder + "/tree_reg_x" + str(count_f) + "_.csv", delimiter=",")
        loaded_data_y = np.loadtxt(csv_folder + "/tree_reg_y" + str(count_f) + "_.csv", delimiter=",")

        if(count_f <= 1):
            stacked_data_x = np.copy(loaded_data_x)
            stacked_data_y = np.copy(loaded_data_y)
        else:
            stacked_data_x = np.vstack([stacked_data_x,loaded_data_x])
            stacked_data_y = np.vstack([stacked_data_y,loaded_data_y])


        print('Loaded file no:' + str(count_f))
    else:
        break; 




Loaded file no:1
Loaded file no:2
Loaded file no:3
Loaded file no:4


In [3]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor,HistGradientBoostingRegressor
from sklearn.metrics import mean_squared_error
from sklearn.multioutput import MultiOutputRegressor

X_train, X_test, y_train, y_test = train_test_split(stacked_data_x, 
                                                    stacked_data_y, 
                                                    test_size=0.01, random_state=42)


In [4]:

# Initialize and train the random forest regressor
rf_regressor = RandomForestRegressor(n_estimators=12, 
                                     n_jobs=cpus_reg, verbose =1, max_depth=4)
#odel = MultiOutputRegressor(rf_regressor)
#rf_regressor = RandomForestRegressor(verbose =1, max_depth=2)
rf_regressor = MultiOutputRegressor(rf_regressor,n_jobs = cpus_reg)
rf_regressor.fit(X_train, y_train)

# Predict on the testing set
y_pred = rf_regressor.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)
    
        #tf.reset_default_graph()
#for index,model in enumerate(cloned_models):
#history_model = model.fit(train_dataset
#                          , epochs=no_of_epochs,
#                          callbacks=[model_checkpoint_callback],
#                          )


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs

In [ ]:
import pickle


# Serialize the object
with open(model_path, 'wb') as outp:  # 'wb' for writing in binary mode
    pickle.dump(rf_regressor, outp, pickle.HIGHEST_PROTOCOL)


In [ ]:
print(y_test.shape)
print(y_pred.shape)

(1, 2622)
(1, 2622)


In [ ]:
40 --> 1 tree -->max=depth = 6 --> 5 GB -- > 10000 line --> 15 min
40 --> 2 tree -->max=depth = 6 --> 6 GB -- > 10000 line --> 3 min

SyntaxError: invalid syntax (2737355240.py, line 1)